In [1]:
pip install keras-tuner

^C
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder() 
from sklearn.preprocessing import OneHotEncoder
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict,cross_val_score
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense, LSTM, Embedding
from keras.models import Sequential
import os
import matplotlib.pyplot as plt
import re


from keras.models import Sequential
from keras.layers import Dense

from keras.layers import Embedding
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation,GlobalMaxPooling1D

from keras import backend as K
from kerastuner import HyperModel
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

from tensorflow.keras.callbacks import EarlyStopping
from keras import regularizers
from tensorflow.keras.layers import BatchNormalization

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from numpy import dstack
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

C:\Users\hager\AppData\Local\Temp/ipykernel_10976/1235258861.py:37: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import HyperModel


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
word2vemodel = gensim.models.Word2Vec.load('/content/drive/MyDrive/ArabicSentimentHager/full_grams_cbow_300_twitter.mdl')

In [ ]:
#read train dataset
train= pd.read_csv('/content/drive/MyDrive/ArabicSentimentHager/paper2/Deep/sg/dataset1/Training_Main-AHS.csv',encoding='utf-8')
train=train.dropna()

y_train1 = train.label                           
X_train= train.drop('label',axis = 1 )

train.label.value_counts()

In [ ]:
#read testing dataset
test= pd.read_csv('/content/drive/MyDrive/ArabicSentimentHager/paper2/Deep/sg/dataset1/Testing_Main-AHS.csv',encoding='utf-8')
test=test.dropna()

y_test1= test.label                           
X_test= test.drop('label',axis = 1 )
test.label.value_counts()

In [ ]:
#convert label to categorical
encoder_train_y = LabelEncoder()
y_train= encoder_train_y.fit_transform(y_train1)
y_train_categorical = np_utils.to_categorical(y_train)

encoder_test_y = LabelEncoder()
y_test = encoder_test_y.fit_transform(y_test1)
y_test_categorical=np_utils.to_categorical(y_test)

In [ ]:
MAX_LEN = 140  # cut texts after this number of words (among top max_features most common words)
BATCH_SIZE = 2000
MAX_NUM_WORDS = 200000
EMBEDDING_DIM = 300
epoch=20


tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(X_train['text'])
train_sequences = tokenizer.texts_to_sequences(X_train['text'])

word_index = tokenizer.word_index
padded_train = pad_sequences(sequences=train_sequences, maxlen=MAX_LEN)

test_sequences = tokenizer.texts_to_sequences(X_test['text'])
padded_test = pad_sequences(sequences=test_sequences, maxlen=MAX_LEN)

print('Total unique tokens generated: ',len(word_index))
print('Shape of padded train tensor: ', padded_train.shape)
print('Shape of padded test tensor: ', padded_test.shape)

In [ ]:
num_words = min(MAX_NUM_WORDS, len(word_index))
word_embedding_matrix = np.zeros((num_words + 1, EMBEDDING_DIM))

for word, index in word_index.items():
    if index > MAX_NUM_WORDS:
        continue
    # embedding_vector = model[word]
    #word = clean_str(word)
    if word not in word2vemodel.wv:
        embedding_vector = None
    else:
        embedding_vector = word2vemodel.wv[word]
    if embedding_vector is not None:
        word_embedding_matrix[index] = embedding_vector

In [ ]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
from keras.models import load_model

model_DEEPCNN= load_model('/content/drive/MyDrive/ArabicSentimentHager/paper2/Deep/CBOW/dataset1/model/DeepCNN.h5', custom_objects={'f1_m': f1_m,  'precision_m': precision_m, 'recall_m': recall_m} )
model_CNN_LSTM= load_model('/content/drive/MyDrive/ArabicSentimentHager/paper2/Deep/CBOW/dataset1/model/CNNLSTM.h5', custom_objects={'f1_m': f1_m,  'precision_m': precision_m, 'recall_m': recall_m} )
model_CNN_GRU= load_model('/content/drive/MyDrive/ArabicSentimentHager/paper2/Deep/CBOW/dataset1/model/CNN_GRU.h5', custom_objects={'f1_m': f1_m,  'precision_m': precision_m, 'recall_m': recall_m} )


In [ ]:
members={'DEEPCNN':model_DEEPCNN,'CNN_LSTM':model_CNN_LSTM, 'CNN_GRU':model_CNN_GRU}

In [ ]:
members['DEEPCNN'].summary()

In [ ]:
'''for key in members:
    for i in range(0, len(members[key].layers)-1):
        members[key].layers[i].trainable = False'''

In [ ]:
epoch=30
BATCH_SIZE=1000

# create stacked model input dataset as outputs from the ensemble
def stacked_dataset(members,padded_train, padded_test, y_train_categorical,stack_Train, stack_Test,BATCH_SIZE,epoch):
    for key in members:     
        
        if key=='DEEPCNN':
            members['DEEPCNN'].fit(padded_train, y_train_categorical, epochs=epoch, batch_size=BATCH_SIZE)
            yhat1 = members['DEEPCNN'].predict(padded_train, verbose=0)
            yhat2 = members['DEEPCNN'].predict(padded_test, verbose=0)
            
        elif key=='CNN_LSTM':
            members['CNN_LSTM'].fit(padded_train, y_train_categorical, epochs=epoch, batch_size=BATCH_SIZE)
            yhat1 = members['CNN_LSTM'].predict(padded_train, verbose=0)
            yhat2 = members['CNN_LSTM'].predict(padded_test, verbose=0)
        elif key=='CNN_GRU':

            members['CNN_GRU'].fit(padded_train, y_train_categorical, epochs=epoch, batch_size=BATCH_SIZE)
            yhat1 = members['CNN_GRU'].predict(padded_train,verbose=0)
            yhat2 = members['CNN_GRU'].predict(padded_test,verbose=0)
            
        
        if stack_Train is None:
            stack_Train = yhat1
        else:
            stack_Train = dstack((stack_Train, yhat1))
        
        if stack_Test is None:
            stack_Test = yhat2
        else:
            stack_Test = dstack((stack_Test, yhat2))
            
    stack_Train = stack_Train.reshape((stack_Train.shape[0], stack_Train.shape[1]*stack_Train.shape[2]))
    stack_Test = stack_Test.reshape((stack_Test.shape[0], stack_Test.shape[1]*stack_Test.shape[2]))
    return stack_Train, stack_Test




In [ ]:
for key in members:
    for i in range(0, len(members[key].layers)-1):
        members[key].layers[i].trainable = False

In [ ]:
from sklearn.model_selection import StratifiedKFold

SFold = StratifiedKFold(n_splits=10,  shuffle=True,random_state=10)

In [ ]:
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate

from tensorflow.keras.backend import clear_session

scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score, average='weighted'),
           'recall' : make_scorer(recall_score, average='weighted'), 
           'f1_score' : make_scorer(f1_score, average='weighted')}


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.model_selection import cross_validate

In [ ]:
SVM_DIC={'Accurcy':[], 'Precision': [],'Recall':[],'F1':[]}
LR_DIC={'Accurcy':[], 'Precision': [],'Recall':[],'F1':[]}
RF_DIC={'Accurcy':[], 'Precision': [],'Recall':[],'F1':[]}
SVM_DIC_Train={'Accurcy':[], 'Precision': [],'Recall':[],'F1':[],'SDAccurcy':[], 'SDPrecision': [],'SDRecall':[],'SDF1':[]}
LR_DIC_Train={'Accurcy':[], 'Precision': [],'Recall':[],'F1':[], 'SDAccurcy':[], 'SDPrecision': [],'SDRecall':[],'SDF1':[]}
RF_DIC_Train={'Accurcy':[], 'Precision': [],'Recall':[],'F1':[], 'SDAccurcy':[], 'SDPrecision': [],'SDRecall':[],'SDF1':[]}

for i in range(0,1):


    SFold = StratifiedKFold(n_splits=10,  shuffle=True,random_state=120)
    stackX_Test=None
    stackX_Train=None
    stackX_Train,stackX_Test =stacked_dataset(members,padded_train, padded_test, y_train_categorical, stackX_Train, stackX_Test,BATCH_SIZE, epoch)
    
    
    param_grid = {'C': [6,7,8,9,10,11,12,1,2,3,4,5]}
    grid_search = GridSearchCV(estimator = SVC(), param_grid = param_grid, cv = SFold) 
    grid_search.fit(stackX_Train, y_train)

    
    model_SVM = grid_search.best_estimator_     
    
    scores_Train_SVM = cross_validate(model_SVM, stackX_Train, y_train, scoring=scoring,  cv=SFold)
   
    yhat_SVM=cross_val_predict(model_SVM,stackX_Test,y_test, cv=SFold)

    SVM_DIC['Accurcy'].append(round(100*accuracy_score(y_test, yhat_SVM), 2))
    SVM_DIC['Precision'].append(round(100*precision_score(y_test, yhat_SVM,average='weighted'), 2))
    SVM_DIC['Recall'].append(round(100*recall_score(y_test, yhat_SVM, average='weighted'), 2))
    SVM_DIC['F1'].append(round(100*f1_score(y_test, yhat_SVM, average='weighted') , 2))     
    
    
   

In [ ]:
print("SVM Test")
ReultofTest=pd.DataFrame([])
ReultofTest=ReultofTest.append({'AccuracyTest' : round(np.mean(SVM_DIC['Accurcy']),2),
                                'PrecisionTest':round(np.mean(SVM_DIC['Precision']),2),
             'RecallTest' : round(np.mean(SVM_DIC['Recall']),2),'F1Test':round(np.mean(SVM_DIC['F1']),2)}, ignore_index=True)

ReultofTest.reindex(['AccuracyTest','PrecisionTest','RecallTest','F1Test'], axis=1)